In [19]:
from notion_client import Client
import re
from getpass import getpass

In [11]:
notion = Client(auth=getpass("Notion auth token: "))
PARENT_PAGE_ID = getpass("Parent page id: ")  

Notion auth token:  ········
Parent page id:  ········


In [21]:
FILE_PATH = "running_plan.md"

with open(FILE_PATH, 'r', encoding='utf-8') as file:
    markdown_content = file.read()

In [22]:
title_match = re.search(r'^# (.+)$', markdown_content, re.MULTILINE)
title = title_match.group(1) if title_match else "Running Plan"

In [23]:
def parse_markdown_text(text):

    bold_pattern = r'\*\*([^*]+)\*\*'
    parts = re.split(r'(\*\*[^*]+\*\*)', text)
    rich_text = []

    for part in parts:
        if re.match(bold_pattern, part):
            rich_text.append({
                "type": "text",
                "text": {"content": part[2:-2]},  # strip the "**"
                "annotations": {"bold": True}
            })
        elif part:
            rich_text.append({
                "type": "text",
                "text": {"content": part}
            })

    return rich_text

In [24]:
def markdown_to_notion_blocks(content):
    
    blocks = []
    lines = content.split('\n')
    
    i = 0
    while i < len(lines):
        line = lines[i].strip()
        
        if not line:  # Skip empty lines
            i += 1
            continue

        # Markdown Heading (### Week X)
        heading3_match = re.match(r'^### (.+)$', line)
        if heading3_match:
            blocks.append({
                "object": "block",
                "type": "heading_3",
                "heading_3": {
                    "rich_text": [{"type": "text", "text": {"content": heading3_match.group(1)}}]
                }
            })
            i += 1
            continue

        # Training day with bold format: * **Day X**: Activity
        day_match = re.match(r'^\* \*\*(Day \d+)\*\*: (.+)$', line)
        if day_match:
            full_text = f"**{day_match.group(1)}**: {day_match.group(2)}"
            blocks.append({
                "object": "block",
                "type": "to_do",
                "to_do": {
                    "rich_text": parse_markdown_text(full_text),
                    "checked": False
                }
            })
            i += 1
            continue

        # Alt format without bold: * Day X: Activity
        alt_day_match = re.match(r'^\* (Day \d+): (.+)$', line)
        if alt_day_match:
            full_text = f"**{alt_day_match.group(1)}**: {alt_day_match.group(2)}"
            blocks.append({
                "object": "block",
                "type": "to_do",
                "to_do": {
                    "rich_text": parse_markdown_text(full_text),
                    "checked": False
                }
            })
            i += 1
            continue

        # Generic asterisk bullet, maybe with bold
        list_match = re.match(r'^\* (.+)$', line)
        if list_match:
            blocks.append({
                "object": "block",
                "type": "to_do",
                "to_do": {
                    "rich_text": parse_markdown_text(list_match.group(1)),
                    "checked": False
                }
            })
            i += 1
            continue

        # Dash bullet
        dash_match = re.match(r'^- (.+)$', line)
        if dash_match:
            blocks.append({
                "object": "block",
                "type": "to_do",
                "to_do": {
                    "rich_text": parse_markdown_text(dash_match.group(1)),
                    "checked": False
                }
            })
            i += 1
            continue

        # Checkbox format: - [x] Activity
        todo_match = re.match(r'^- \[([ xX])\] (.+)$', line)
        if todo_match:
            checked = todo_match.group(1).lower() == 'x'
            blocks.append({
                "object": "block",
                "type": "to_do",
                "to_do": {
                    "rich_text": parse_markdown_text(todo_match.group(2)),
                    "checked": checked
                }
            })
            i += 1
            continue

        # Fallback: plain paragraph
        blocks.append({
            "object": "block",
            "type": "paragraph",
            "paragraph": {
                "rich_text": parse_markdown_text(line)
            }
        })
        i += 1
    
    return blocks


In [25]:
blocks = markdown_to_notion_blocks(markdown_content)

In [26]:
def create_notion_page():
    try:
        # Create the page
        page = notion.pages.create(
            parent={"page_id": PARENT_PAGE_ID},
            properties={
                "title": {
                    "title": [{"text": {"content": title}}]
                }
            }
        )
        
        notion.blocks.children.append(
            block_id=page["id"],
            children=blocks
        )
        
        print(f"✅ Created Notion page: {title}")
        print(f"🔗 URL: {page.get('url')}")
        
        todo_count = sum(1 for block in blocks if block["type"] == "to_do")
        checked_count = sum(1 for block in blocks if block["type"] == "to_do" and block["to_do"]["checked"])
        
        print(f"📋 To-do items: {todo_count} ({checked_count} completed)")
        
        return page
    
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return None

In [27]:
result = create_notion_page()

✅ Created Notion page: Running Plan
🔗 URL: https://www.notion.so/Running-Plan-1eed001cc4148127b84cc5c01117a590
📋 To-do items: 32 (0 completed)
